# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [83]:
document = ET.parse( './data/mondial_database.xml' )

In [42]:
#1. 10 countries with the lowest infant mortality rates
names, mortalities = [], []
for country in document.iterfind('country'):
    name = country.find('name').text
    try:
        mort = country.find('infant_mortality').text
    except:
        mort = np.nan
    names.append(name)
    mortalities.append(mort)
morts = pd.Series(mortalities, index=names)
morts = morts[morts.notnull()].sort_values()
morts.head(10)


Monaco                   1.81
Romania                 10.16
Fiji                     10.2
Brunei                  10.48
Grenada                  10.5
Mauritius               10.59
Panama                   10.7
Seychelles              10.77
United Arab Emirates    10.92
Barbados                10.93
dtype: object

In [43]:
#2. 10 cities with the largest population
names, populations = [], []
for city in document.iter('city'):
    name = city.find('name').text
    population = city.findall('population')
    for i in population:
        if i.attrib['year'] == '2011':
            pop = int(i.text)
    populations.append(pop)
    names.append(name)

pops = pd.Series(populations, names)
pops.sort_values(ascending=False).head(10)

Mumbai       12442373
Delhi        11034555
Hubli         8443675
Bangalore     8443675
Mysore        8443675
London        8250205
Tehran        8154051
Dhaka         7423137
Thimphu       6731790
Warangal      6731790
dtype: int64

In [44]:
#3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
country_name, ethnicity, population = [], [], []
for country in document.iterfind('country'):
    name = country.find('name').text
    populations = country.findall('population')
    for i in populations:
        if i.attrib['year'] == '2011':
            pop = int(i.text)
    eth = country.findall('ethnicgroup')
    for e in eth:
        country_name.append(name)
        ethnicity.append(e.text)
        population.append(float(e.attrib['percentage'])/100*pop)
data = {'country_names': country_name, 'ethnicgroups': ethnicity, 'population': population}
df = pd.DataFrame(data)
ethnic = df.groupby('ethnicgroups').sum()
ethnic.sort_values('population', ascending=False).head(10)

,population
ethnicgroups,
Indo-Aryan,"871,815,583.44"
African,"667,724,568.37"
Bhote,"605,427,488.50"
Nepalese,"423,799,241.95"
Dravidian,"302,713,744.25"
Bengali,"146,776,916.72"
Burman,"101,845,207.52"
Arab,"93,474,053.28"
Arab-Berber,"85,709,405.06"


In [155]:
#Create dataframe for country name to code joins
country_name, car_codes = [], []
for country in document.iterfind('country'):
    name = country.find('name').text
    car_code = country.attrib['car_code']
    country_name.append(name)
    car_codes.append(car_code)
countries_temp = {'country_name': country_name, 'car_code': car_codes}
countries = pd.DataFrame(countries_temp)

In [156]:
#4. name and country of a) longest river, b) largest lake and c) airport at highest elevation
river_name, river_length, car_code = [], [], []
for r in document.iterfind('river'):
    name = r.find('name').text
    try:
        length = float(r.find('length').text)
    except:
        length = np.nan
    counts = r.attrib['country'].split(" ")
    for i in counts:        
        river_name.append(name)
        river_length.append(length)
        car_code.append(i)
rivers_temp = {'river_name': river_name, 'length': river_length, 'car_code': car_code}
rivers = pd.DataFrame(rivers_temp)
rivers_df = pd.merge(rivers, countries)
print('Longest Rivers - for some reason the Nile\'s length isn\'t in the XML')
rivers_df.sort_values('length', ascending=False).head(10)

Longest Rivers - for some reason the Nile's length isn't in the XML


,car_code,length,river_name,country_name
310,PE,"6,448.00",Amazonas,Peru
299,CO,"6,448.00",Amazonas,Colombia
302,BR,"6,448.00",Amazonas,Brazil
226,CN,"6,380.00",Jangtse,China
225,CN,"4,845.00",Hwangho,China
43,R,"4,400.00",Lena,Russia
364,ZRE,"4,374.00",Zaire,Zaire
390,RCB,"4,374.00",Zaire,Congo
227,CN,"4,350.00",Mekong,China
246,LAO,"4,350.00",Mekong,Laos


In [157]:
#4. name and country of a) longest river, b) largest lake and c) airport at highest elevation
lake_name, lake_area, car_code = [], [], []
for r in document.iterfind('lake'):
    name = r.find('name').text
    try:
        area = float(r.find('area').text)
    except:
        area = np.nan
    country = r.attrib['country'].split(' ')
    for i in country:
        lake_name.append(name)
        lake_area.append(area)
        car_code.append(i)
lakes_temp = {'lake_name': lake_name, 'area': lake_area, 'car_code': car_code}
lakes = pd.DataFrame(lakes_temp)
lakes = pd.merge(lakes, countries)
print('Largest Lakes')
lakes.sort_values('area', ascending=False).head(10)

Largest Lakes


,area,car_code,lake_name,country_name
56,"386,400.00",R,Caspian Sea,Russia
72,"386,400.00",IR,Caspian Sea,Iran
73,"386,400.00",AZ,Caspian Sea,Azerbaijan
74,"386,400.00",KAZ,Caspian Sea,Kazakhstan
77,"386,400.00",TM,Caspian Sea,Turkmenistan
148,"82,103.00",CDN,Lake Superior,Canada
154,"82,103.00",USA,Lake Superior,United States
116,"68,870.00",EAT,Lake Victoria,Tanzania
104,"68,870.00",EAK,Lake Victoria,Kenya
108,"68,870.00",EAU,Lake Victoria,Uganda


In [158]:
#4. name and country of a) longest river, b) largest lake and c) airport at highest elevation
airport_name, airport_elevation, car_code = [], [], []
for r in document.iterfind('lake'):
    name = r.find('name').text
    try:
        elevation = float(r.find('elevation').text)
    except:
        elevation = np.nan
    country = r.attrib['country']
    airport_name.append(name)
    airport_elevation.append(elevation)
    car_code.append(country)
airports_temp = {'airport_name': airport_name, 'elevation': airport_elevation, 'car_code': car_code}
airports = pd.DataFrame(airports_temp)
airports = pd.merge(airports, countries)
print('Highest Airport')
airports.sort_values('elevation', ascending=False).head(10)

Highest Airport


,airport_name,car_code,elevation,country_name
48,Nam Co,CN,"4,718.00",China
97,Poopo,BOL,"3,686.00",Bolivia
98,Salar de Uyuni,BOL,"3,650.00",Bolivia
45,Koli Sarez,TAD,"3,250.00",Tajikistan
94,Lake Irazu,CR,"3,200.00",Costa Rica
47,Qinghai Lake,CN,"3,195.00",China
51,Segara Anak,RI,"2,008.00",Indonesia
85,Lake Tahoe,USA,"1,900.00",United States
83,Crater Lake,USA,"1,883.00",United States
59,Lake Tana,ETH,"1,830.00",Ethiopia
